# Video understanding - content moderation for social media videos
Social media video content moderation with Video Understanding leverages AI to automatically detect unsafe or inappropriate content, such as violence, nudity, or harmful behavior, across large volumes of user-generated videos. By analyzing both audio and visual signals, it enables platforms to maintain community standards, reduce manual review effort, and protect audiences at scale.

In this sample notebook, we use metadata extracted with the Video Understanding tool, which includes shot-level moderation results and audio transcripts with timestamps. We will retrieve this metadata and apply LLMs for deeper moderation analysis.
![content_moderation](./statics/video-content-moderation.png)

In [ ]:
from utils import s3_tool, bedrock_tool
import json
import boto3
from IPython.display import JSON, Markdown, display

The results of the Video Understanding Tool are stored in both Amazon S3 and Amazon DynamoDB, and you can access them directly within the same AWS account.

In this lab, we will retrieve the extracted video summaries and audio transcriptions from S3 using the functions in `utils.s3_tool`. The tool stores extraction files in the solution-provisioned S3 bucket, following the naming convention: `bedrock-mm-{account_id}-{region}`.

All extracted information is organized under a `tasks` prefix within the data bucket, with each task’s data stored in a separate sub-prefix corresponding to the task ID. You can review the S3 bucket to explore all the extracted information.

In [ ]:
# Print out your data bucket
session = boto3.session.Session()
sts = session.client("sts")
account_id = sts.get_caller_identity()["Account"]
region = session.region_name

print(f"bedrock-mm-{account_id}-{region}")

You can find the task ID in the Video Understanding Tool UI.

Go to the Shot-based section and select the content moderation sample video that was processed earlier using the UI. Click the Get Task ID link then replace the value below with it.

![Find task Id](./statics/find-task-id-moderation.png)


In [ ]:
task_id = 'YOUR_TASK_ID_FROM_VIDEO_UNDERSTANDING_TOOL'

Retrieve transcripts from the Video Understanding Tool's managed database.

In [ ]:
transcripts = s3_tool.get_transcripts(task_id)
print(json.dumps(transcripts,indent=2))

Retrieve shot summaries from the Video Understanding Tool's managed database.

In [ ]:
shots = s3_tool.get_shot_outputs(task_id=task_id, output_names=["Summarize shot","Content Moderation","Detect text"])
print(json.dumps(shots,indent=2))

## Analyze the shot moderation detection results and the 
For content moderation use cases, you can customize prompts to analyze each video shot or frame and assign moderation labels for easier decision-making. In this sample, we ask the LLM to consider both visual and audio inputs to make an informed decision about whether the video is unsafe.

In [ ]:
prompt = f'''
You are a content moderation expert. Your task is to determine whether a video is safe for audiences by analyzing both the shot-level moderation labels and the audio transcripts.
Evaluate the video against the following categories:
- Alcohol: Scenes or audio references involving alcohol consumption.
- Tobacco: Scenes or audio references involving tobacco or smoking.
- Suggestive: Content such as nudity, barechested males, revealing clothing, or other sexually suggestive material.
- Toxicity: Offensive or harmful language in the audio, including hate speech, harassment, or explicit language.

Instructions:
Consider both the visual (shot-level moderation labels) and audio (transcripts).
Clearly state whether the video contains any of the above categories.

Shot Summary and moderation labels:
{shots}
Audio Transcript:
{transcripts}
'''

In [ ]:
response = bedrock_tool.bedrock_converse(
    model_id="us.amazon.nova-pro-v1:0", 
    prompt=prompt, 
    inference_config={
      "maxTokens": 10000,
      "topP": 0.1,
      "temperature": 0.3
    }
)
result = bedrock_tool.parse_converse_response(response)

In [ ]:
from IPython.display import Markdown, display
display(Markdown(result.replace('\\n', '\n').replace('\\"', '"')))